In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd /content/


/content


In [4]:
cd drive/


/content/drive


In [5]:
cd My Drive

/content/drive/My Drive


In [6]:
cd Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [7]:
cd dw_matrix/

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [8]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [10]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [11]:
mean_price = np.mean( df['prices_amountmin'])
mean_price

93.53952188183217

In [12]:
y_true =df['prices_amountmin']
y_pred = [mean_price] * y_true.shape[0]

mean_absolute_error(y_true, y_pred)

63.968341227870624

In [13]:
df.brand.value_counts()


Nike                       1709
PUMA                        574
Ralph Lauren                524
New Balance                 361
VANS                        356
                           ... 
Mcnett                        1
Shape Form                    1
Croft & Barrow (Kohl's)       1
TAILIAN                       1
DeRoyal                       1
Name: brand, Length: 1856, dtype: int64

In [0]:

df ['brand_cat']= df ['brand'].factorize()[0]

In [15]:

feats = ['brand_cat']
X = df[feats].values
y = df['prices_amountmin'].values

model= DecisionTreeRegressor(max_depth=5)

scores=cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-58.38655694633361, 4.223555478221712)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X= df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [17]:
df['brand_cat'] = df['brand'].map(lambda x:str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [18]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model), 

((-57.31783843165656, 4.181246596160967),)

In [19]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
      
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value


  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [21]:
keys = set()


df['features_parsed'].map( lambda x: keys.update(x.keys()) )
len(keys)

476

In [22]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [23]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_chronograph', 'feat_fastener type', 'feat_insulation',
       'feat_lens width', 'feat_protection', 'feat_weight', 'feat_date',
       'feat_product dimensions', 'feat_sub type', 'feat_is portable'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
 keys_stat[key]= df[ False==  df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [25]:
{k:v for k,v in keys_stat.items() if v > 35}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [27]:
run_model(['feat_brand_cat'])

(-61.79232381882688, 3.7460167781236606)

In [28]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[  df.brand == df.feat_brand ].shape


(8846, 1002)

In [0]:
feats = ['']


In [30]:
model = RandomForestRegressor(max_depth= 5, n_estimators = 100)
run_model(['brand_cat'], model)

(-57.318987456509106, 4.2069117675257885)

In [31]:
feats_cat = [x for x in df.columns if 'cat' in x ]
feats_cat
      

['categories',
 'brand_cat',
 'feat_fabrication',
 'feat_location - country',
 'feat_catalog',
 'feat_certifications and listings',
 'feat_location - city/state',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_clothing category',
 'feat_shoe category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_adjustable_cat',
 'feat_auto_cat',
 'feat_spikes type_cat',
 'feat_fabric care instructions_cat',
 'feat_is wheeled_cat',
 'feat_features & fastening_cat',
 'feat_case diameter_cat',
 'feat_style name_cat',
 'feat_picture_cat',
 'feat_what includes_cat',
 'feat_hat style_cat',
 'feat_size/dimensions_cat',
 'feat_lens_cat',
 'feat_black_cat',
 'feat_fabric content_cat',
 'feat_dial color_cat',
 'feat_country////region of manufacture_cat',
 'feat_impact resistant_cat',
 'feat_charger included_cat',
 'feat_foot arch_cat',
 'feat_part type_cat',
 'feat_rise_cat'

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat', 'feat_sport_cat']
#feats += feats_cat
#feats = list(set(feats))
model = RandomForestRegressor(max_depth= 5, n_estimators = 100)
result= run_model(feats , model)

In [37]:
X= df [feats].values
y =df['prices_amountmin'].values

m = RandomForestRegressor(max_depth= 5, n_estimators = 100, random_state= 0)
m.fit(X,y)

print(result)
perm =PermutationImportance(m, random_state= 1).fit(X, y);
eli5.show_weights(perm, feature_names = feats)

(-57.1074273062247, 4.1711350376962475)


Weight,Feature
0.2631 ± 0.0101,brand_cat
0.1087 ± 0.0093,feat_material_cat
0.0436 ± 0.0027,feat_gender_cat
0.0219 ± 0.0012,feat_brand_cat
0.0002 ± 0.0001,feat_sport_cat


In [34]:
df['brand'].value_counts()

nike            1777
puma             609
ralph lauren     526
vans             386
new balance      371
                ... 
covona             1
sport wrap         1
surplus            1
sabatier           1
nat nast           1
Name: brand, Length: 1732, dtype: int64

In [62]:
df [ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'football', 'condition': 'new with box'}, {},
       {'sport': 'football, lacrosse', 'type': 'cleats', 'condition': 'new without box'},
       {},
       {'sport': 'football', 'condition': 'new with box', 'type': 'cleats'}],
      dtype=object)

In [35]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [41]:
!git add matrix_one/day5.ipynb
!git commit -m "day5"
!git push

fatal: pathspec 'matrix_one/day5.ipynb' did not match any files
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   matrix_one/day3.ipynb
	modified:   matrix_one/day4.ipynb

no changes added to commit
Everything up-to-date


In [0]:
 !git config --global user.email "j.nadolski@me.com"
 !git config --global user.name "jaro19853"